In [1]:
import numpy as np
import pandas as pd
from sklearn.externals.joblib import load, dump
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, SpatialDropout1D, Convolution1D, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from common import SEED, TARGETS

c:\users\alberto\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
VOCABULARY_SIZE = 100000
SEQ_MAX_LEN = 100
EMBEDDINGS_FILE = "glove.840B.300d.txt"
EMBEDDINGS_SIZE = 300
EMBEDDINGS_CACHE = "cache/embeddings.pkl"
WEIGHTS_CACHE = "cache/cnn_glove_weights.hdf5"
VALIDATION_PRED_FILE = "cache/cnn_glove_validation_pred_fold_%s.pkl"
SUBMISSION_FILE = "submissions/submission_cnn_glove.csv.gz"
K_FOLDS = 10

In [3]:
np.random.seed(SEED)
np.set_printoptions(suppress=True)

In [4]:
from tensorflow.python.client import device_lib

def get_available_devices():  
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:GPU:0']


In [5]:
train = pd.read_csv("data/train_clean.csv", encoding="utf-8")
test = pd.read_csv("data/test_clean.csv", encoding="utf-8")
submission = pd.read_csv("data/sample_submission.csv", encoding="utf-8")

In [6]:
%%time
word_tokenizer = text.Tokenizer(num_words=VOCABULARY_SIZE)
word_tokenizer.fit_on_texts(train.comment_text.values)

Wall time: 9.52 s


In [7]:
%%time
list_tokenized_word_train = word_tokenizer.texts_to_sequences(train.comment_text.values)
list_tokenized_word_test = word_tokenizer.texts_to_sequences(test.comment_text.values)

Wall time: 13 s


In [8]:
%%time
x_train_word_sequences = sequence.pad_sequences(list_tokenized_word_train, maxlen=SEQ_MAX_LEN)
x_test_word_sequences = sequence.pad_sequences(list_tokenized_word_test, maxlen=SEQ_MAX_LEN)

Wall time: 2.19 s


In [9]:
def load_embeddings():
    try:
        return load(EMBEDDINGS_CACHE)
    except IOError:
        embeddings_index = {}
        f = open(EMBEDDINGS_FILE, 'r', encoding="utf-8", errors="ignore")
        for index, line in enumerate(f):
            try:
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype="float32")
                embeddings_index[word] = coefs
            except Exception:
                print("Unable to parse line %d, skipping" % index)
        f.close()
        dump(embeddings_index, EMBEDDINGS_CACHE)
        return embeddings_index

In [10]:
%%time
embeddings_index = load_embeddings()
print("Loaded %s word vectors" % len(embeddings_index))

Loaded 2195884 word vectors
Wall time: 1min 47s


In [11]:
def compute_embedding_matrix(embeddings_index, word_index):
    embedding_matrix = np.zeros((VOCABULARY_SIZE, EMBEDDINGS_SIZE))
    for word, i in word_index.items():
        if i >= VOCABULARY_SIZE:
            break
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [12]:
%%time
embedding_matrix = compute_embedding_matrix(embeddings_index, word_tokenizer.word_index)
print(embedding_matrix.shape)

(100000, 300)
Wall time: 208 ms


In [13]:
def build_network(embedding_matrix):
    nn = Sequential()
    nn.add(Embedding(VOCABULARY_SIZE, EMBEDDINGS_SIZE, weights=[embedding_matrix], trainable=False, input_length=SEQ_MAX_LEN))
    nn.add(SpatialDropout1D(0.3))
    nn.add(Convolution1D(120, 3, padding="valid", activation="relu", strides=1))
    nn.add(GlobalMaxPooling1D())
    nn.add(Dense(120, activation="sigmoid"))
    nn.add(Dropout(0.5))
    nn.add(Dense(6, activation="sigmoid"))
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return nn

In [14]:
kfold = KFold(n_splits=K_FOLDS, random_state=SEED)

In [15]:
scores = []
test_preds = []

In [16]:
for index, (train_index, test_index) in enumerate(kfold.split(x_train_word_sequences, train[TARGETS].values)):
    
    x_train, x_val = x_train_word_sequences[train_index], x_train_word_sequences[test_index]
    y_train, y_val = train[TARGETS].values[train_index], train[TARGETS].values[test_index]
    
    callbacks = [
        EarlyStopping(monitor="val_loss"),
        ModelCheckpoint(filepath=WEIGHTS_CACHE, save_best_only=True)
    ]
    
    network = build_network(embedding_matrix)
    history = network.fit(
        x_train, y_train,
        epochs=100, batch_size=32, validation_data=(x_val, y_val), verbose=1, callbacks=callbacks
    )
    
    network.load_weights(WEIGHTS_CACHE)
    
    y_pred = network.predict_proba(x_val)
    dump(y_pred, VALIDATION_PRED_FILE % index)
    
    score = roc_auc_score(y_val, y_pred)
    print("ROC-AUC score: %0.4f" % score)
    scores.append(score)
    
    test_preds.append(network.predict_proba(x_test_word_sequences))

Train on 143613 samples, validate on 15958 samples
Epoch 1/100
143613/143613 [==============================] - 105s 730us/step - loss: 0.0619 - acc: 0.9784 - val_loss: 0.0514 - val_acc: 0.9797
Epoch 2/100
143613/143613 [==============================] - 96s 670us/step - loss: 0.0497 - acc: 0.9813 - val_loss: 0.0449 - val_acc: 0.9823
Epoch 3/100
143613/143613 [==============================] - 93s 648us/step - loss: 0.0474 - acc: 0.9820 - val_loss: 0.0462 - val_acc: 0.9815
ROC-AUC score: 0.9841
Train on 143614 samples, validate on 15957 samples
Epoch 1/100
143614/143614 [==============================] - 95s 663us/step - loss: 0.0618 - acc: 0.9784 - val_loss: 0.0506 - val_acc: 0.9809
Epoch 2/100
143614/143614 [==============================] - 94s 653us/step - loss: 0.0495 - acc: 0.9815 - val_loss: 0.0484 - val_acc: 0.9808
Epoch 3/100
143614/143614 [==============================] - 93s 651us/step - loss: 0.0469 - acc: 0.9823 - val_loss: 0.0476 - val_acc: 0.9814
Epoch 4/100
143614/1436

In [18]:
print("Average ROC-AUC: %0.4f" % np.mean(scores))

Average ROC-AUC: 0.9838


In [19]:
for index, target in enumerate(TARGETS):
    y = 0
    for fold in range(0, K_FOLDS):
        y = y + test_preds[fold][:, index]
    submission[target] = y / K_FOLDS

In [20]:
# Scores 0.9807
submission.to_csv(SUBMISSION_FILE, index=False, encoding="utf-8", compression="gzip")